In [1]:
import pandas as pd
import os

# Get data for data discovery

In [2]:
folder_processed_data_path = './processed_data/'
processed_website_data_folder = './processed_data/website/'

We import the dataframe computed in preprocessing.

In [3]:
# import the name by movie
print("name_by_movie_ordered_pvalue_10_5_df :")
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
# name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)
display(name_by_movie_df.sample(2))
print(f"length : {len(name_by_movie_df)}\n")

# import movie genres
print("movie_genres_df :")
movie_genres_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_genres_df.csv'))
display(movie_genres_df.sample(2))
print(f"length : {len(movie_genres_df)}\n")

# import the movie dataframe
print("movie_df :")
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
# movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df.sample(2))
print(f"length : {len(movie_df)}")

name_by_movie_ordered_pvalue_10_5_df :


,wiki_ID,char_words,order,gender,t_stat,p_value,slope_change
61908,4277184,Hess,3.0,M,NaN,NaN,0.000000
28352,1148228,Katie,2.0,F,-0.609123,0.554815,0.000704


length : 172906

movie_genres_df :


,wiki_ID,genre
4560,144765,Crime Fiction
168232,23953216,Satire


length : 213854

movie_df :


,wiki_ID,mov_name,year,month,revenue,numVotes,averageRating,poster_url,IMDB_ID
48185,14409654,Pirate Radio USA,2005,NaN,NaN,70,8.1,NaN,tt0833984
31753,18247508,Freebird,2008,2.0,NaN,662,5.4,/jHP4WqwHnFYbyRxN2ZdDSPvKTrM.jpg,tt0478154


length : 65503


In [4]:
# keep only the movie_id and name and drop duplicates as we remove the gender column
char_name_per_movie = name_by_movie_df[['wiki_ID', 'char_words']].drop_duplicates()
display(char_name_per_movie.sample(2))
print(f"Length change : {len(name_by_movie_df)} -> {len(char_name_per_movie)}")

,wiki_ID,char_words
158651,29830825,Chandni
116459,18692364,Lefty


Length change : 172906 -> 167984


In [5]:
# add a column with the genre of the movie
char_name_per_movie_with_genre = char_name_per_movie.merge(movie_genres_df, on='wiki_ID', how='left').copy(deep=True)

# rearrange and rename the columns
char_name_per_movie_with_genre = char_name_per_movie_with_genre[['genre', 'char_words', 'wiki_ID']].copy(deep=True)
char_name_per_movie_with_genre.rename(columns={'char_words': 'char_name', 'wiki_ID': 'movie_id'}, inplace=True)

display(char_name_per_movie_with_genre.sample(2))
print(f"Length change : {len(char_name_per_movie)} -> {len(char_name_per_movie_with_genre)}") # the increase is due to the fact that some movies have multiple genres

,genre,char_name,movie_id
139226,Romantic comedy,Raeford,894764
272237,Horror,Hank,2756655


Length change : 167984 -> 739925


In [6]:
# for each name and genre, count the number of movies
movie_count_per_name_and_genre = char_name_per_movie_with_genre.groupby(['genre', 'char_name']).count().reset_index()
movie_count_per_name_and_genre.rename(columns={'movie_id': 'nb_movies'}, inplace=True)
display(movie_count_per_name_and_genre.sample(10))
print(f"Length of movie_count_per_name_and_genre : {len(movie_count_per_name_and_genre)}")

,genre,char_name,nb_movies
235833,Thriller,Carey,4
193871,Romance Film,Maire,1
172965,New Hollywood,Laughlin,1
55974,Comedy,Kong,4
35871,Black comedy,Giles,1
95948,Drama,Anna,212
127503,Film adaptation,El,1
71101,Computer Animation,Rosetta,2
30836,Biographical film,Riel,1
16407,Adventure,Debra,3


Length of movie_count_per_name_and_genre : 254789


In [7]:
# drop the non relevant name 'The' from the dataframe
movie_count_per_name_and_genre = movie_count_per_name_and_genre[movie_count_per_name_and_genre['char_name'] != 'The'].copy(deep=True)

# List of irrelevant names
irrelevant_names = ['The', 'Agent', 'Man', 'Girl', 'General', 'Young', 'Chief', 'Captain']

# Filter rows where 'char_name' is not in the list of irrelevant names
movie_count_per_name_and_genre = movie_count_per_name_and_genre[~movie_count_per_name_and_genre['char_name'].isin(irrelevant_names)].copy(deep=True)


In [8]:
# For each genre, keep only the 10 names with the most counts
movie_count_per_name_and_genre.reset_index(inplace=True)
movie_count_per_name_and_genre.sort_values(by=['genre', 'nb_movies'], ascending=False, inplace=True)
top10_per_genre = movie_count_per_name_and_genre.groupby('genre').head(10).reset_index(drop=True)
top10_per_genre.set_index(['genre', 'char_name'], inplace=True)
display(top10_per_genre)

index  nb_movies
genre       char_name                   
Zombie Film Alice      254325          6
            Frank      254486          5
            Sheriff    254731          5
            Ben        254358          4
            Doctor     254447          4
...                       ...        ...
Absurdism   George        176          4
            King          271          4
            Roger         414          4
            Thompson      476          4
            Alan            7          3

[3145 rows x 2 columns]

In [9]:
# keep only the rows with a genre among the 5 genres with the highest number of names
top5_genres = movie_count_per_name_and_genre.groupby('genre').sum().sort_values(by='nb_movies', ascending=False).head(5).index
top10_name_top5_genre = top10_per_genre.loc[top5_genres, :].copy(deep=True)
top10_name_top5_genre.drop(columns=['index'], inplace=True)
print(top10_name_top5_genre)

                        nb_movies
genre        char_name           
Drama        John             701
             Jack             426
             Mary             379
             Joe              338
             Paul             334
             Frank            333
             George           325
             David            324
             Tom              307
             James            280
Comedy       John             261
             Jack             249
             Joe              193
             Frank            187
             George           186
             Sam              181
             Mary             180
             Tom              165
             Bob              161
             Paul             152
Thriller     John             304
             Jack             246
             Frank            183
             James            152
             David            151
             Michael          151
             Tom              145
             S

In [10]:
# rename the genre 'Romance Film' to 'Romance'
top10_name_top5_genre.reset_index(inplace=True)
top10_name_top5_genre.loc[top10_name_top5_genre['genre'] == 'Romance Film', 'genre'] = 'Romance'
top10_name_top5_genre.set_index(['genre', 'char_name'], inplace=True)
display(top10_name_top5_genre.sample(2))

,,nb_movies
genre,char_name,
Thriller,David,151
Action,Tom,101


Let's compute the percentages. First lets compute the total number of movie per genre

In [11]:
display(char_name_per_movie_with_genre)

# count the number of movie per genre
nb_movies_per_genre = char_name_per_movie_with_genre.reset_index().groupby('genre')['movie_id'].count().to_frame()
nb_movies_per_genre.columns = ['total_nb_movies']

# replace the genre "Romance Film" by "Romance" in the nb_movies_per_genre dataframe
nb_movies_per_genre.reset_index(inplace=True)
nb_movies_per_genre.loc[nb_movies_per_genre['genre'] == 'Romance Film', 'genre'] = 'Romance'
nb_movies_per_genre.set_index('genre', inplace=True)
display(nb_movies_per_genre)

# add total number of movies per genre in a new column of char_name_per_movie_with_genre
top10_name_top5_genre_with_percent = top10_name_top5_genre.reset_index().merge(nb_movies_per_genre, on='genre', how='left').copy(deep=True)

# compute the percentage of movies per genre for each name and add it in a new column of top10_name_top5_genre
top10_name_top5_genre_with_percent['percentage'] = top10_name_top5_genre_with_percent['nb_movies'] / top10_name_top5_genre_with_percent['total_nb_movies']
display(top10_name_top5_genre_with_percent)


,genre,char_name,movie_id
0,Action,Gold,3217
1,Comedy,Gold,3217
2,Time travel,Gold,3217
3,Black comedy,Gold,3217
4,Zombie Film,Gold,3217
...,...,...,...
739920,Comedy film,Ajay,37478048
739921,Drama,Murphy,37501922
739922,Drama,Hunter,37501922
739923,Drama,John,37501922


,total_nb_movies
genre,
Absurdism,712
Acid western,29
Action,30904
Action Comedy,959
Action Thrillers,2704
...,...
Women in prison films,51
Workplace Comedy,641
World cinema,14972


,genre,char_name,nb_movies,total_nb_movies,percentage
0,Drama,John,701,93752,0.007477
1,Drama,Jack,426,93752,0.004544
2,Drama,Mary,379,93752,0.004043
3,Drama,Joe,338,93752,0.003605
4,Drama,Paul,334,93752,0.003563
5,Drama,Frank,333,93752,0.003552
6,Drama,George,325,93752,0.003467
7,Drama,David,324,93752,0.003456
8,Drama,Tom,307,93752,0.003275
9,Drama,James,280,93752,0.002987


In [12]:
# drop the total_nb_movies column
top10_name_top5_genre_with_percent.drop(columns=['total_nb_movies'], inplace=True)
top10_name_top5_genre_with_percent.set_index(['genre', 'char_name'], inplace=True)
display(top10_name_top5_genre_with_percent)

nb_movies  percentage
genre    char_name                       
Drama    John             701    0.007477
         Jack             426    0.004544
         Mary             379    0.004043
         Joe              338    0.003605
         Paul             334    0.003563
         Frank            333    0.003552
         George           325    0.003467
         David            324    0.003456
         Tom              307    0.003275
         James            280    0.002987
Comedy   John             261    0.004930
         Jack             249    0.004703
         Joe              193    0.003645
         Frank            187    0.003532
         George           186    0.003513
         Sam              181    0.003419
         Mary             180    0.003400
         Tom              165    0.003117
         Bob              161    0.003041
         Paul             152    0.002871
Thriller John             304    0.007801
         Jack             246    0.006313
         Frank            183    0.004696
         James            152    0.003901
         David            151    0.003875
         Michael          151    0.003875
         Tom              145    0.003721
         Sam              141    0.003618
         Paul             130    0.003336
         Mike             127    0.003259
Romance  John             233    0.006349
         Jack             146    0.003978
         Mary             142    0.003869
         Paul             129    0.003515
         Joe              127    0.003460
         George           125    0.003406
         David            117    0.003188
         Frank            113    0.003079
         Tom              112    0.003052
         Sam              111    0.003024
Action   John             252    0.008154
         Jack             187    0.006051
         Frank            149    0.004821
         James            147    0.004757
         Joe              105    0.003398
         Tom              101    0.003268
         Sam               96    0.003106
         Sheriff           91    0.002945
         Michael           90    0.002912
         Jim               85    0.002750

In [13]:
# max_values_per_genre = top10_name_top5_genre.reset_index().groupby('genre')['nb_movies'].max()
# min_values_per_genre = top10_name_top5_genre.reset_index().groupby('genre')['nb_movies'].min()

# # merge the two series into one dataframe
# max_min_per_genre = pd.concat([max_values_per_genre, min_values_per_genre], axis=1)
# max_min_per_genre.columns = ['max', 'min']

# # merge this dataframe with the top10_name_top5_genre dataframe
# top10_name_top5_genre = top10_name_top5_genre.reset_index().merge(max_min_per_genre, on='genre', how='left')

# # perform a normalization of the number of movies to 0 to 1000
# top10_name_top5_genre['nb_movies_normalized'] = (top10_name_top5_genre['nb_movies'] - top10_name_top5_genre['min']) / (top10_name_top5_genre['max'] - top10_name_top5_genre['min']) * 1000
# display(top10_name_top5_genre)

In [14]:
# top10_name_top5_genre.set_index(['genre', 'char_name'], inplace=True)
# top10_name_top5_genre.drop(columns=['max', 'min'], inplace=True)
# top10_name_top5_genre.rename(columns={'nb_movies_normalized': 'scaled'}, inplace=True)
# display(top10_name_top5_genre)

# Export the dataframes

In [15]:
# Export top10_name_top5_genre.csv
display(top10_name_top5_genre_with_percent.reset_index().sample(2))
top10_name_top5_genre_with_percent.to_csv(os.path.join(processed_website_data_folder, 'top10_name_top5_genre.csv'), index=True)

,genre,char_name,nb_movies,percentage
11,Comedy,Jack,249,0.004703
29,Thriller,Mike,127,0.003259
